In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85

In [3]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 5- Machine Learning Algorithms with HP tuning")

<Experiment: artifact_location='s3://campusxalidvcbucket/425591744193892905', creation_time=1755800992412, experiment_id='425591744193892905', last_update_time=1755800992412, lifecycle_stage='active', name='Exp 5- Machine Learning Algorithms with HP tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import mlflow
import mlflow.sklearn
import optuna

In [5]:
df = pd.read_csv('reddit_preprocessing.csv')

In [ ]:
#Step2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Fill NaN values in 'clean_comment' with empty strings
df['clean_comment'] = df['clean_comment'].fillna('')

ngram_range = (1,3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features=max_features)
X=vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state = 42)
X_resampled, y_resampled = smote.fit_resample(X, y)

#Step3: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)


#Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
  with mlflow.start_run():
     # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


#Step5: Optuna Objective function for XGBoost
def objective_svm(trial):
  C = trial.suggest_float('C', 1e-4, 10.0, log=True)
  kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

  #LogisticRegression model setup with balanced class weight
  # Change solver to 'lbfgs' which supports l2 penalty with sparse data
  model = SVC(C=C, kernel=kernel, random_state=42)
  return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


#Step7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
  study = optuna.create_study(direction = 'maximize')
  study.optimize(objective_svm, n_trials = 15)

  #Get the best parameters and log only the best model
  best_params = study.best_params
  # Change solver to 'lbfgs' which supports l2 penalty with sparse data
  best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

  #log the model
  log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

run_optuna_experiment()

[I 2025-08-22 03:46:42,838] A new study created in memory with name: no-name-f6cf1226-4523-42e1-8ae3-77489ce1bdf2
[I 2025-08-22 03:49:16,062] Trial 0 finished with value: 0.4036774807143612 and parameters: {'C': 0.00014299306921510577, 'kernel': 'linear'}. Best is trial 0 with value: 0.4036774807143612.
[I 2025-08-22 03:51:44,839] Trial 1 finished with value: 0.4038888301807038 and parameters: {'C': 0.0016221519585109413, 'kernel': 'linear'}. Best is trial 1 with value: 0.4038888301807038.
[I 2025-08-22 03:54:21,735] Trial 2 finished with value: 0.4263975483461904 and parameters: {'C': 0.01837268613305761, 'kernel': 'poly'}. Best is trial 2 with value: 0.4263975483461904.
[I 2025-08-22 04:00:21,147] Trial 3 finished with value: 0.8302863785268942 and parameters: {'C': 1.7413279590355977, 'kernel': 'rbf'}. Best is trial 3 with value: 0.8302863785268942.
[I 2025-08-22 04:02:39,954] Trial 4 finished with value: 0.7869597379266617 and parameters: {'C': 0.29938854889707417, 'kernel': 'rbf'}